In [1]:
import numpy as np
import pandas as pd

from utils import load_doc_vectors

doc_vectors = load_doc_vectors()
df = pd.DataFrame(doc_vectors)
df.fillna(0, inplace=True)

ModuleNotFoundError: No module named 'configs'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=6, test_size=.20)

acc = {}
mse = {}

# k nearest neighbors for k = {1, 3, 5}
for k in [1, 3, 5]:
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(x_train, y_train)
    predictions = knn.predict(x_test)
    acc[f'knn{k}'] = accuracy_score(y_test, predictions)
    mse[f'knn{k}'] = mean_squared_error(y_test, predictions)
# multinomial naive bayes classifier
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
predictions = mnb.predict(x_test)
acc['mnb'] = accuracy_score(y_test, predictions)
mse['mnb'] = mean_squared_error(y_test, predictions)

# MSE comparison of models
ax = pd.DataFrame(mse, index=(1,)).plot(kind='bar', figsize=(10, 5))
ax.set_ylabel('Mean Square Error')
ax.set_xlabel('Models')
plt.show()